# PageRank

In [7]:
def f(x):
    print x
    list1=[]
    s=len(x[1][0])
    print x[1][0], x[1][1], s
    for y in x[1][0]:
        list1.append(tuple((y,x[1][1]/s)))
    return list1

def p(x):
    print x

In [8]:
list=[('A',('D',)),('B',('A',)),('C',('A','B')),('D',('A','C'))]
pages=sc.parallelize(list).map(lambda x:(x[0],tuple(x[1]))).partitionBy(2).cache()#必须转换成key-values,持久化操作提高效率，partitionBy将相同key的元素哈希到相同的机器上，省去了后续join操作shuffle开销
links=sc.parallelize(['A','B','C','D']).map(lambda x:(x,0.25))
print pages.collect()
print links.collect()

rank=pages.join(links)
print rank.collect()

print rank.flatMap(f).collect()

[('A', ('D',)), ('C', ('A', 'B')), ('B', ('A',)), ('D', ('A', 'C'))]
[('A', 0.25), ('B', 0.25), ('C', 0.25), ('D', 0.25)]
[('A', (('D',), 0.25)), ('D', (('A', 'C'), 0.25)), ('C', (('A', 'B'), 0.25)), ('B', (('A',), 0.25))]
[('D', 0.25), ('A', 0.125), ('C', 0.125), ('A', 0.125), ('B', 0.125), ('A', 0.25)]


In [9]:
f(('A', (('D',), 0.25)))

('A', (('D',), 0.25))
('D',) 0.25 1


[('D', 0.25)]

In [10]:
f(('D', (('A', 'C'), 0.25)))

('D', (('A', 'C'), 0.25))
('A', 'C') 0.25 2


[('A', 0.125), ('C', 0.125)]

In [11]:
for i in range(1,3):
    print i
    rank=pages.join(links).flatMap(f)
    print rank.collect()
    links=rank.reduceByKey(lambda x, y:x+y)
    print links.collect()
    links=links.mapValues(lambda x:0.05+0.8*x)
    #links.foreach(p)
    print links.collect()
# links.saveAsSequenceFile("pagerank.output")

1
[('D', 0.25), ('A', 0.125), ('C', 0.125), ('A', 0.125), ('B', 0.125), ('A', 0.25)]
[('A', 0.5), ('D', 0.25), ('C', 0.125), ('B', 0.125)]
[('A', 0.45), ('D', 0.25), ('C', 0.15000000000000002), ('B', 0.15000000000000002)]
2
[('A', 0.15000000000000002), ('A', 0.125), ('C', 0.125), ('D', 0.45), ('A', 0.07500000000000001), ('B', 0.07500000000000001)]
[('B', 0.07500000000000001), ('D', 0.45), ('A', 0.35000000000000003), ('C', 0.125)]
[('B', 0.11000000000000001), ('D', 0.41000000000000003), ('A', 0.33), ('C', 0.15000000000000002)]
